Paquetes necesarios

In [2]:
import cv2  
import math 

from ultralytics import YOLO
from matplotlib import pyplot as plt
import easyocr

Extraemos las clases del modelo YOLO 11

In [ ]:
model = YOLO('yolo11n.pt')

vid = cv2.VideoCapture("C0142.MP4")

names = None

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        results = model(frame, show=False)
        if names is None:
            names = results[0].names
        annotated_frame = results[0].plot()
        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()

# Leemos las posibles clases
with open("classes.txt", "w") as f:
    f.write(str(names))

### Mostramos el funcionamiento de nuestro modelo entrenado

In [ ]:
model = YOLO('best.pt')

vid = cv2.VideoCapture("C0142.MP4")

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        results = model(frame, show=False)
        annotated_frame = results[0].plot()
        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()

### Usamos el modelo pre-entrenado de YOLO y el nuestro en conjunto 
Utilizamos el modelo pre-entrenado para detectar personas y vehículos, posteriormente, cuando hayamos detectado un vehículo, se lo pasamos a nuestro modelo entrenado en matrículas para que le detecte la matrícula.

In [ ]:
base_model = YOLO('yolo11n.pt')
our_model = YOLO('best.pt')
vid = cv2.VideoCapture("C0142.MP4")

reader = easyocr.Reader(['es'], gpu=False) 

classes = [0, 2, 3, 5, 7]    # Person, car, motorcycle, bus, truck

car_boxes = []
car_boxes_left_coords = []

while vid.isOpened():
    ret, frame = vid.read()

    if ret:
        base_results = base_model(frame, show=False, classes=classes)
        plates_result = None
        annotated_frame = base_results[0].plot()
        bounding_boxes = list()
        names = list()
        for result in base_results:
            bounding_boxes += (result.boxes.xyxy).tolist()
            names += [result.names[cls.item()] for cls in result.boxes.cls.int()]
            for i, name in enumerate(names):
                x1, y1, x2, y2 = [int(item) for item in bounding_boxes[i]]
                if name != "person":
                    vehicle_box = frame[y1:y2, x1:x2]
                    plates_result = our_model(vehicle_box, show=False)
                    if len(plates_result[0].boxes) > 0:
                        plate_detection = (plates_result[0].boxes.xyxy).tolist()
                        px1, py1, px2, py2 = [int(item) for item in plate_detection[0]]
                        plate = vehicle_box[py1:py2, px1:px2]
                        cv2.
                        real_x1 = px1+x1
                        real_y1 = py1+y1
                        real_x2 = px2+x1
                        real_y2 = py2+y1
                        res = reader.readtext(plate)
                        for (bbox, text, prob) in res:
                            (top_left, top_right, bottom_right, bottom_left) = bbox
                            cv2.putText(frame, text, (real_x1, real_x2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                        cv2.rectangle(annotated_frame, (real_x1, real_y1), (real_x2, real_y2), (0, 255, 0), 1)
                        
        cv2.imshow("Deteccion de YOLO", annotated_frame)

        # Salir del vídeo cuando presionamos ESC
        if cv2.waitKey(1) & 0xFF == 27 or cv2.getWindowProperty("Deteccion de YOLO", cv2.WND_PROP_VISIBLE) < 1:
            break
    else:
        # El vídeo ya se terminó
        break

vid.release()
cv2.destroyAllWindows()

Using CPU. Note: This module is much faster with a GPU.



0: 384x640 4 cars, 1 bus, 62.1ms
Speed: 36.7ms preprocess, 62.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 544x640 (no detections), 92.2ms
Speed: 3.4ms preprocess, 92.2ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

0: 512x640 (no detections), 142.8ms
Speed: 2.8ms preprocess, 142.8ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)

0: 416x640 (no detections), 105.8ms
Speed: 1.9ms preprocess, 105.8ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 (no detections), 64.7ms
Speed: 2.0ms preprocess, 64.7ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)

0: 480x640 (no detections), 110.8ms
Speed: 2.7ms preprocess, 110.8ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 4 cars, 1 bus, 98.3ms
Speed: 3.5ms preprocess, 98.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 512x640 (no detections), 99.1ms
Speed: 4.2ms preprocess, 99.1m

c:\Users\lllrm\anaconda3\envs\VC_P4_easyOCR\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


0: 448x640 (no detections), 99.8ms
Speed: 2.6ms preprocess, 99.8ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 81.9ms
Speed: 2.8ms preprocess, 81.9ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 640x608 (no detections), 81.2ms
Speed: 3.0ms preprocess, 81.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 608)

0: 448x640 (no detections), 60.3ms
Speed: 1.4ms preprocess, 60.3ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 (no detections), 65.1ms
Speed: 2.3ms preprocess, 65.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 608x640 (no detections), 77.6ms
Speed: 3.8ms preprocess, 77.6ms inference, 0.6ms postprocess per image at shape (1, 3, 608, 640)

0: 416x640 (no detections), 65.9ms
Speed: 2.0ms preprocess, 65.9ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

0: 448x640 (no detections), 71.2ms
Speed: 2.7ms preprocess, 71.2ms in

In [ ]:

base_results = base_model(frame, show=False, classes=classes)
plates_result = None
annotated_frame = base_results[0].plot()
for box in result.boxes:
    if result.names[int(box.cls)] != "person":
        x1, y1, x2, y2 = box.xyxy[0].int().tolist()
        car_boxes.append(frame[y1:y2, x1:x2])
        car_boxes_left_coords.append((x1, y1))
if car_boxes:
    plates_results = our_model(car_boxes, show=False)
    for i, plates_result in enumerate(plates_results):
        car_x, car_y = car_boxes_left_coords[i]
        if len(plates_result.boxes) > 0:
            px1, py1, px2, py2 = plates_result.boxes[0].xyxy[0].int().tolist()
            real_x1 = px1 + car_x
            real_y1 = py1 + car_y
            real_x2 = px2 + car_x
            real_y2 = py2 + car_y
            cv2.rectangle(annotated_frame, (real_x1, real_y1), (real_x2, real_y2), (0, 255, 0), 3)


    